# SGR Quality: Executive Notebook (C-Level)

Цель: на одном экране показать **сильные кейсы**, **негативные кейсы** и **почему это важно бизнесу**.

- Позитивные кейсы: `final_score_for_message >= 0.90` и `judge_label = 1`.
- Негативные кейсы: строго **только последний успешный run** (`latest_run`, `judge_label = 0`).
- Бизнес-вид: понятные русские поля без перегруза ID.


In [ ]:
from __future__ import annotations

import sqlite3
from pathlib import Path

import pandas as pd

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 240)


def resolve_db_path() -> Path:
    cwd = Path.cwd().resolve()
    candidates = [cwd / "dialogs.db", cwd.parent / "dialogs.db"]
    candidates.extend(parent / "dialogs.db" for parent in cwd.parents)

    for path in candidates:
        if path.exists():
            return path

    raise FileNotFoundError("dialogs.db not found. Run: make init-fresh && make scan")


DB_PATH = resolve_db_path()
conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row


def qdf(sql: str, params: tuple[object, ...] = ()) -> pd.DataFrame:
    return pd.read_sql_query(sql, conn, params=params)


def latest_run_id() -> str:
    row = conn.execute(
        """
        SELECT run_id
        FROM scan_runs
        WHERE status = 'success'
        ORDER BY started_at_utc DESC
        LIMIT 1
        """
    ).fetchone()
    if row is None:
        raise ValueError("No successful scan run found. Run: make scan")
    return str(row["run_id"])


def as_yes_no(value: object) -> str:
    if pd.isna(value):
        return "N/A"
    return "Да" if bool(int(value)) else "Нет"


def as_pct(value: object) -> str:
    if pd.isna(value):
        return "N/A"
    return f"{float(value):.1%}"


def style_business_table(df: pd.DataFrame, caption: str | None = None):
    if df.empty:
        print("Таблица пустая для выбранного условия.")
        return

    styler = (
        df.style.hide(axis="index")
        .set_properties(
            **{
                "white-space": "pre-wrap",
                "overflow-wrap": "anywhere",
                "word-break": "break-word",
                "vertical-align": "top",
                "text-align": "left",
            }
        )
        .set_table_styles(
            [
                {
                    "selector": "th",
                    "props": [
                        ("text-align", "left"),
                        ("white-space", "normal"),
                        ("background-color", "#f4f6f8"),
                    ],
                },
                {
                    "selector": "caption",
                    "props": [
                        ("caption-side", "top"),
                        ("text-align", "left"),
                        ("font-weight", "bold"),
                    ],
                },
            ]
        )
    )

    if caption:
        styler = styler.set_caption(caption)

    display(styler)


NEGATIVE_RECOMMENDATIONS = {
    "greeting": "Начинайте сообщение с короткого персонального приветствия, чтобы сразу зафиксировать контакт.",
    "upsell": "Добавляйте следующий платный шаг, релевантный контексту клиента, а не только скидку или статус.",
    "empathy": "Явно признавайте ситуацию клиента одной фразой перед предложением решения.",
}

RUN_ID = latest_run_id()
print(f"Используется последний успешный run_id: {RUN_ID}")
print(f"Используется база: {DB_PATH}")
print("Проверка визуала: display.max_colwidth =", pd.get_option("display.max_colwidth"))
assert pd.get_option("display.max_colwidth") is None


## 1) Что происходит в последнем запуске

Ниже KPI-карточки по **latest run**: объем проверки, доля подтвержденных оценок и масштаб хороших/негативных кейсов.


In [ ]:
run_snapshot = qdf(
    """
    WITH latest_run AS (
        SELECT run_id
        FROM scan_runs
        WHERE status = 'success'
        ORDER BY started_at_utc DESC
        LIMIT 1
    )
    SELECT
        run_id,
        model,
        selected_conversations,
        messages_count,
        started_at_utc,
        finished_at_utc,
        json_extract(summary_json, '$.processed') AS processed,
        json_extract(summary_json, '$.inserted') AS inserted,
        json_extract(summary_json, '$.judged') AS judged,
        json_extract(summary_json, '$.skipped_due_to_errors') AS skipped_due_to_errors
    FROM scan_runs
    WHERE run_id = (SELECT run_id FROM latest_run)
    """
)

kpi = qdf(
    """
    WITH latest_run AS (
        SELECT run_id
        FROM scan_runs
        WHERE status = 'success'
        ORDER BY started_at_utc DESC
        LIMIT 1
    ),
    base AS (
        SELECT
            sr.message_id,
            sr.judge_label
        FROM scan_results sr
        JOIN messages m ON m.message_id = sr.message_id
        WHERE sr.run_id = (SELECT run_id FROM latest_run)
          AND m.speaker_label = 'Sales Rep'
    ),
    scored AS (
        SELECT
            b.*,
            AVG(CASE WHEN b.judge_label = 1 THEN 1.0 ELSE 0.0 END)
                OVER (PARTITION BY b.message_id) AS final_score
        FROM base b
    )
    SELECT
        COUNT(*) AS total_rule_checks,
        SUM(CASE WHEN judge_label IS NOT NULL THEN 1 ELSE 0 END) AS judged_checks,
        SUM(CASE WHEN judge_label = 1 THEN 1 ELSE 0 END) AS judge_confirmed_checks,
        SUM(CASE WHEN judge_label = 0 THEN 1 ELSE 0 END) AS negative_checks,
        COUNT(DISTINCT CASE WHEN final_score >= 0.90 THEN message_id END) AS strong_messages,
        COUNT(DISTINCT message_id) AS evaluated_messages,
        CASE
            WHEN SUM(CASE WHEN judge_label IS NOT NULL THEN 1 ELSE 0 END) = 0 THEN NULL
            ELSE 1.0 * SUM(CASE WHEN judge_label = 1 THEN 1 ELSE 0 END)
                 / SUM(CASE WHEN judge_label IS NOT NULL THEN 1 ELSE 0 END)
        END AS judge_correctness
    FROM scored
    """
)

cards = pd.DataFrame(
    [
        {"Показатель": "Run ID", "Значение": str(run_snapshot.loc[0, "run_id"])},
        {"Показатель": "Модель", "Значение": str(run_snapshot.loc[0, "model"])},
        {"Показатель": "Проверок по правилам", "Значение": int(kpi.loc[0, "total_rule_checks"])},
        {"Показатель": "Проверок с вердиктом judge", "Значение": int(kpi.loc[0, "judged_checks"])},
        {"Показатель": "Доля подтвержденных оценок", "Значение": as_pct(kpi.loc[0, "judge_correctness"])},
        {"Показатель": "Сильных сообщений (>=0.90)", "Значение": int(kpi.loc[0, "strong_messages"])},
        {"Показатель": "Негативных проверок", "Значение": int(kpi.loc[0, "negative_checks"])},
    ]
)

style_business_table(cards, caption="KPI latest run")

run_view = run_snapshot.rename(
    columns={
        "selected_conversations": "Выбрано диалогов",
        "messages_count": "Сообщений в диапазоне",
        "started_at_utc": "Старт (UTC)",
        "finished_at_utc": "Завершение (UTC)",
        "processed": "Обработано кейсов",
        "inserted": "Сохранено результатов",
        "judged": "Проставлено judge",
        "skipped_due_to_errors": "Пропущено из-за ошибок",
    }
)
style_business_table(run_view.drop(columns=["run_id", "model"]), caption="Снимок запуска")


## 2) Сильные кейсы (почему хорошо)

Контракт отбора (SQL):
- берем только `Sales Rep`;
- считаем `final_score` по сообщению;
- оставляем `final_score >= 0.90` и `judge_label = 1`.

Ниже показывается **полная таблица** (не `head()`) с длинными текстами без обрезки.


In [ ]:
good_cases = qdf(
    """
    WITH latest_run AS (
        SELECT run_id
        FROM scan_runs
        WHERE status = 'success'
        ORDER BY started_at_utc DESC
        LIMIT 1
    ),
    base AS (
        SELECT
            sr.message_id,
            m.text AS message,
            sr.rule_key,
            sr.eval_hit,
            sr.judge_expected_hit,
            sr.judge_label,
            sr.eval_confidence,
            sr.judge_confidence,
            sr.eval_reason_code,
            sr.eval_reason,
            sr.judge_rationale,
            sr.evidence_quote
        FROM scan_results sr
        JOIN messages m ON m.message_id = sr.message_id
        WHERE sr.run_id = (SELECT run_id FROM latest_run)
          AND m.speaker_label = 'Sales Rep'
    ),
    scored AS (
        SELECT
            b.*,
            AVG(CASE WHEN b.judge_label = 1 THEN 1.0 ELSE 0.0 END)
                OVER (PARTITION BY b.message_id) AS final_score,
            SUM(CASE WHEN b.judge_label = 1 THEN 1 ELSE 0 END)
                OVER (PARTITION BY b.message_id) AS good_rules,
            COUNT(*) OVER (PARTITION BY b.message_id) AS total_rules
        FROM base b
    )
    SELECT
        s.message,
        s.rule_key AS rule,
        s.eval_hit AS evaluator_hit,
        s.judge_expected_hit AS expected_hit_by_judge,
        ROUND(s.eval_confidence, 3) AS eval_conf,
        ROUND(s.judge_confidence, 3) AS judge_conf,
        s.eval_reason_code,
        s.eval_reason AS evaluator_why,
        s.judge_rationale AS judge_why,
        s.evidence_quote AS quote_from_message,
        ROUND(s.final_score, 3) AS final_score_for_message,
        (s.good_rules || '/' || s.total_rules) AS good_rules_for_message
    FROM scored s
    WHERE s.final_score >= 0.90
      AND s.judge_label = 1
    ORDER BY s.final_score DESC, s.judge_confidence DESC, s.message, s.rule_key
    """
)

print(f"Найдено сильных проверок: {len(good_cases)}")

good_business = good_cases.rename(
    columns={
        "message": "Сообщение продавца",
        "rule": "Правило (Rule)",
        "evaluator_hit": "Решение evaluator",
        "expected_hit_by_judge": "Ожидание judge",
        "eval_conf": "Уверенность evaluator",
        "judge_conf": "Уверенность judge",
        "eval_reason_code": "Код причины evaluator",
        "evaluator_why": "Почему evaluator так решил",
        "judge_why": "Почему judge согласен/не согласен",
        "quote_from_message": "Дословная цитата из сообщения",
        "final_score_for_message": "Итоговая оценка сообщения",
        "good_rules_for_message": "Сколько правил выполнено",
    }
)

if not good_business.empty:
    good_business["Решение evaluator"] = good_business["Решение evaluator"].map(as_yes_no)
    good_business["Ожидание judge"] = good_business["Ожидание judge"].map(as_yes_no)

style_business_table(good_business, caption="Сильные кейсы: полная таблица latest run")


## 3) Негативные кейсы (что исправить)

Источник: **только последний успешный run** (`latest_run`) и `judge_label = 0`.

Ниже показываются понятные для бизнеса поля и отдельная колонка `Рекомендация для бизнеса`.


In [ ]:
negative_cases = qdf(
    """
    WITH latest_run AS (
        SELECT run_id
        FROM scan_runs
        WHERE status = 'success'
        ORDER BY started_at_utc DESC
        LIMIT 1
    )
    SELECT
        sr.conversation_id,
        sr.message_id,
        m.text AS message,
        sr.rule_key,
        sr.eval_hit,
        sr.judge_expected_hit,
        ROUND(sr.eval_confidence, 3) AS eval_conf,
        ROUND(sr.judge_confidence, 3) AS judge_conf,
        sr.eval_reason_code,
        sr.eval_reason,
        sr.judge_rationale,
        sr.evidence_quote
    FROM scan_results sr
    JOIN messages m ON m.message_id = sr.message_id
    WHERE sr.run_id = (SELECT run_id FROM latest_run)
      AND m.speaker_label = 'Sales Rep'
      AND sr.judge_label = 0
    ORDER BY COALESCE(sr.judge_confidence, 0) DESC, sr.message_id, sr.rule_key
    """
)

print(f"Негативных проверок в latest run: {len(negative_cases)}")

if negative_cases.empty:
    print("Негативных кейсов не найдено в последнем успешном запуске.")
else:
    negative_business = negative_cases.rename(
        columns={
            "message": "Сообщение продавца",
            "rule_key": "Правило (Rule)",
            "eval_hit": "Решение evaluator",
            "judge_expected_hit": "Ожидание judge",
            "eval_conf": "Уверенность evaluator",
            "judge_conf": "Уверенность judge",
            "eval_reason_code": "Код причины evaluator",
            "eval_reason": "Почему evaluator так решил",
            "judge_rationale": "Почему judge согласен/не согласен",
            "evidence_quote": "Дословная цитата из сообщения",
        }
    ).copy()

    negative_business["Решение evaluator"] = negative_business["Решение evaluator"].map(as_yes_no)
    negative_business["Ожидание judge"] = negative_business["Ожидание judge"].map(as_yes_no)
    negative_business["Рекомендация для бизнеса"] = negative_business["Правило (Rule)"].map(NEGATIVE_RECOMMENDATIONS).fillna(
        "Разобрать кейс вручную и обновить playbook команды продаж."
    )

    negative_business = negative_business[
        [
            "Сообщение продавца",
            "Правило (Rule)",
            "Решение evaluator",
            "Ожидание judge",
            "Уверенность evaluator",
            "Уверенность judge",
            "Код причины evaluator",
            "Почему evaluator так решил",
            "Почему judge согласен/не согласен",
            "Дословная цитата из сообщения",
            "Рекомендация для бизнеса",
        ]
    ]

    style_business_table(negative_business, caption="Негативные кейсы latest run: полная таблица")


## 4) Глоссарий бизнес-полей (RU + EN)

Этот блок нужен, чтобы C-level и команда аналитики читали метрики одинаково.


In [ ]:
glossary = pd.DataFrame(
    [
        {"Поле": "judge_correctness", "RU": "Доля оценок evaluator, подтвержденных judge", "EN": "Evaluator correctness rate"},
        {"Поле": "judge_label", "RU": "Вердикт judge: evaluator корректен (1) или нет (0)", "EN": "Judge verdict"},
        {"Поле": "judge_expected_hit", "RU": "Какой hit judge считает правильным", "EN": "Judge expected hit"},
        {"Поле": "eval_hit", "RU": "Решение evaluator по правилу", "EN": "Evaluator hit decision"},
        {"Поле": "eval_reason", "RU": "Объяснение evaluator", "EN": "Evaluator rationale"},
        {"Поле": "judge_rationale", "RU": "Объяснение judge", "EN": "Judge rationale"},
        {"Поле": "evidence_quote", "RU": "Дословная цитата из сообщения", "EN": "Evidence quote"},
        {"Поле": "final_score_for_message", "RU": "Итоговый score сообщения по подтвержденным правилам", "EN": "Final message score"},
        {"Поле": "good_rules_for_message", "RU": "Количество подтвержденных правил из общего числа", "EN": "Passed rules ratio"},
    ]
)

style_business_table(glossary, caption="Глоссарий метрик и полей")


## 5) Technical Appendix (для инженеров и аудита)

Здесь намеренно оставлены ID и технические поля для дебага.


In [ ]:
appendix_neg = qdf(
    """
    WITH latest_run AS (
        SELECT run_id
        FROM scan_runs
        WHERE status = 'success'
        ORDER BY started_at_utc DESC
        LIMIT 1
    )
    SELECT
        sr.conversation_id,
        sr.message_id,
        sr.rule_key,
        sr.eval_hit,
        sr.judge_expected_hit,
        sr.judge_label,
        sr.eval_confidence,
        sr.judge_confidence,
        sr.eval_reason_code,
        sr.eval_reason,
        sr.judge_rationale,
        sr.evidence_quote,
        sr.evidence_message_id,
        sr.evidence_span_start,
        sr.evidence_span_end
    FROM scan_results sr
    JOIN messages m ON m.message_id = sr.message_id
    WHERE sr.run_id = ?
      AND m.speaker_label = 'Sales Rep'
      AND sr.judge_label = 0
    ORDER BY sr.message_id, sr.rule_key
    """,
    (RUN_ID,),
)

llm_trace = qdf(
    """
    SELECT phase,
           COUNT(*) AS calls,
           SUM(CASE WHEN error_message <> '' THEN 1 ELSE 0 END) AS errors,
           ROUND(AVG(latency_ms), 1) AS avg_latency_ms,
           MAX(latency_ms) AS max_latency_ms
    FROM llm_calls
    WHERE run_id = ?
    GROUP BY phase
    ORDER BY phase
    """,
    (RUN_ID,),
)

print("Appendix: негативные кейсы с ID и evidence span")
if appendix_neg.empty:
    print("В latest run нет негативных кейсов для технического разбора.")
else:
    display(appendix_neg)

print("\nAppendix: сводка по LLM вызовам")
if llm_trace.empty:
    print("Для latest run не найдено записей в llm_calls.")
else:
    display(llm_trace)


In [ ]:
conn.close()
